In [ ]:
# # STEP 1: Install required packages (if not already installed)
# !pip install -q sentence-transformers==2.2.2 transformers==4.31.0 torch==2.0.1 huggingface_hub==0.16.4 groq nltk

# !pip cache purge
# !rm -rf ~/.cache/huggingface

# STEP 2: Set up
import os
import torch
import numpy as np
from sentence_transformers import SentenceTransformer, util
from nltk.tokenize import sent_tokenize
from groq import Groq
import nltke

# Login to Hugging Face (uncomment if needed)
# !huggingface-cli login–
# Enter your Hugging Face token (get it from https://huggingface.co/settings/tokens)

import numpy as np
# from sentence_transformers import SentenceTransformer, CrossEncoder
from transformers import AutoTokenizer
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.cluster import KMeans
import nltk
import time
import logging
import torch
import os
from groq import Groq
import json

# Set up logging
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')

# Download NLTK data
try:
    nltk.download('punkt', quiet=True)
    nltk.download('punkt_tab')  # Added for sent_tokenize
    logging.info("NLTK data downloaded successfully")
except Exception as e:
    logging.error(f"Failed to download NLTK data: {e}")
    raise

# Verify Groq API key
if not os.environ.get("GROQ_API_KEY"):
    raise EnvironmentError("GROQ_API_KEY environment variable not set")

# Validate input JSON files
reference_json_path = "/kaggle/input/llm-dataset/question_answers.json"
pdf_text_json_path = "/kaggle/input/llm-dataset/pdf_text_data.json"
for path in [reference_json_path, pdf_text_json_path]:
    if not os.path.exists(path):
        logging.error(f"Input file not found: {path}")
        raise FileNotFoundError(f"Input file not found: {path}")

# Check disk usage
!df -h

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.0/86.0 kB 8.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.9/116.9 kB 11.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.4/7.4 MB 123.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 619.9/619.9 MB 2.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.3/13.3 MB 127.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 137.5/137.5 MB 7.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 317.1/317.1 MB 1.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.8/11.8 MB 122.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.0/21.0 MB 107.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 849.3/849.3 kB 52.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 557.1/557.1 MB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 168.4/168.4 MB 7.2 MB/s eta 0:00:00

ImportError: cannot import name 'cached_download' from 'huggingface_hub' (/usr/local/lib/python3.11/dist-packages/huggingface_hub/__init__.py)

In [ ]:
# Check GPU availability
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
logging.info(f"Using device: {device}")

# Set cache directory
cache_dir = "/content/hf_cache"
os.makedirs(cache_dir, exist_ok=True)

# Clear GPU memory
torch.cuda.empty_cache()

# Load embedding model
embedder = SentenceTransformer('all-MiniLM-L6-v2', cache_folder=cache_dir).to(device)

# Load cross-encoder for re-ranking
cross_encoder = CrossEncoder('sentence-transformers/paraphrase-MiniLM-L6-v2', device=device)

# Load tokenizer for token counting
tokenizer = AutoTokenizer.from_pretrained('sentence-transformers/all-MiniLM-L6-v2', cache_dir=cache_dir)

# Initialize Groq client
os.environ["GROQ_API_KEY"] = "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"  # Replace with your actual key
client = Groq(api_key=os.environ["GROQ_API_KEY"])

# Enhanced Groq generation function
def generate_text(messages, max_tokens=1000, retries=3):
    prompt = "\n".join([f"{msg['role']}: {msg['content']}" for msg in messages])
    for attempt in range(retries):
        try:
            completion = client.chat.completions.create(
                model="llama-3.3-70b-versatile",
                messages=messages,
                max_tokens=max_tokens,
                temperature=0.2,  # Slightly higher for natural responses
                top_p=0.9,
                stream=False
            )
            return completion.choices[0].message.content.strip()
        except Exception as e:
            logging.error(f"Groq API error (attempt {attempt+1}/{retries}): {e}")
            time.sleep(1)  # Brief delay before retry
    return "Error: Failed to generate response after multiple attempts"

# Check disk and GPU memory usage
!df -h
!nvidia-smi

In [ ]:
def semantic_chunking(corpus: str, max_sentences_per_chunk: int = 10, min_sentences_per_chunk: int = 3, max_tokens_per_chunk: int = 500) -> list[dict]:
    logging.info("Starting semantic chunking...")
    start_time = time.time()

    # Split into sentences
    sentences = nltk.sent_tokenize(corpus)
    if not sentences:
        logging.warning("Empty corpus provided")
        return []

    # Compute sentence embeddings
    sentence_embeddings = embedder.encode(sentences, show_progress_bar=False)

    # Cluster sentences using KMeans
    num_clusters = max(1, len(sentences) // min_sentences_per_chunk)
    kmeans = KMeans(n_clusters=num_clusters, random_state=42).fit(sentence_embeddings)

    # Group sentences by cluster
    clusters = {}
    for sent, label in zip(sentences, kmeans.labels_):
        if label not in clusters:
            clusters[label] = []
        clusters[label].append(sent)

    # Form chunks from clusters
    chunks = []
    for cluster_sentences in clusters.values():
        for i in range(0, len(cluster_sentences), max_sentences_per_chunk):
            chunk_sentences = cluster_sentences[i:i + max_sentences_per_chunk]
            if len(chunk_sentences) >= min_sentences_per_chunk:
                chunk_text = " ".join(chunk_sentences)
                token_count = len(tokenizer.encode(chunk_text))
                if token_count <= max_tokens_per_chunk:
                    chunks.append({
                        "text": chunk_text,
                        "embedding": embedder.encode(chunk_text),
                        "token_count": token_count
                    })

    logging.info(f"Semantic chunking completed in {time.time() - start_time:.2f} seconds. {len(chunks)} chunks created (avg tokens: {np.mean([c['token_count'] for c in chunks]):.1f})")
    return chunks

In [ ]:
def enhanced_rag_retrieval(query: str, chunks: list[dict], max_tokens: int = 3000, top_k_initial: int = 20, top_k_final: int = 5) -> list[dict]:
    logging.info(f"Retrieving chunks for query: {query}")
    start_time = time.time()

    if not chunks:
        logging.warning("No chunks provided for retrieval")
        return []

    # Initial retrieval with cosine similarity
    query_embedding = embedder.encode(query)
    similarities = cosine_similarity([query_embedding], [c["embedding"] for c in chunks])[0]
    top_k_initial = min(top_k_initial, len(chunks))  # Adjust for small chunk lists
    initial_top_indices = np.argsort(similarities)[::-1][:top_k_initial]

    # Prepare pairs for cross-encoder
    cross_encoder_pairs = [(query, chunks[idx]["text"]) for idx in initial_top_indices]
    if not cross_encoder_pairs:
        return []

    # Re-rank with cross-encoder
    cross_encoder_scores = cross_encoder.predict(cross_encoder_pairs)
    reranked_indices = np.argsort(cross_encoder_scores)[::-1][:min(top_k_final, len(cross_encoder_scores))]
    final_indices = [initial_top_indices[i] for i in reranked_indices]

    # Select chunks within token limit
    selected_chunks = []
    total_tokens = 0
    for idx in final_indices:
        chunk = chunks[idx]
        if total_tokens + chunk["token_count"] <= max_tokens:
            selected_chunks.append(chunk)
            total_tokens += chunk["token_count"]

    logging.info(f"Retrieval completed in {time.time() - start_time:.2f} seconds. Retrieved {len(selected_chunks)} chunks (total tokens: {total_tokens})")
    return selected_chunks

In [ ]:
def normal_rag_answer(query: str, chunks: list[dict], max_tokens: int = 1000) -> str:
    logging.info(f"Generating Normal RAG answer for query: {query}")
    if not chunks:
        logging.warning("No chunks provided for answer generation")
        return "No relevant information found to answer the query."

    # Combine chunk texts
    context = "\n\n".join([c["text"] for c in chunks])
    token_count = len(tokenizer.encode(context))
    logging.info(f"Context token count: {token_count}")

    # High-quality prompt with LLM-as-a-judge style
    prompt = [
        {
            "role": "system",
            "content": (
                "You are an expert assistant tasked with answering questions based on provided context. "
                "Your goal is to generate a concise, accurate, and complete answer using only the relevant information from the context. "
                "If the context is insufficient or irrelevant, state that clearly. "
                "Structure your answer clearly, using bullet points or paragraphs as appropriate, and avoid unnecessary elaboration."
            )
        },
        {
            "role": "user",
            "content": (
                f"Query: {query}\n\n"
                f"Context:\n{context}\n\n"
                "Instructions:\n"
                "1. Analyze the context to identify information directly relevant to the query.\n"
                "2. Provide a complete answer based on the relevant information.\n"
                "3. If the context lacks sufficient details, state: 'The provided context does not contain enough information to answer the query.'\n"
                "4. Keep the answer concise and focused, within {max_tokens} tokens."
            )
        }
    ]

    answer = generate_text(prompt, max_tokens=max_tokens)
    if not answer:
        answer = "Error: Failed to generate answer."
    logging.info(f"Answer generated (length: {len(answer)} characters)")
    return answer

In [ ]:
import json
from typing import List, Dict

def build_corpus_and_chunks(pdf_text: str, max_sentences: int = 10, min_sentences: int = 3, max_tokens: int = 500) -> List[Dict]:
    """Create semantic chunks from PDF text."""
    return semantic_chunking(pdf_text, max_sentences, min_sentences, max_tokens)

def get_rag_answer_from_chunks(question: str, chunks: List[Dict], max_tokens: int = 1000) -> str:
    """Generate RAG answer using retrieved chunks."""
    retrieved = enhanced_rag_retrieval(question, chunks, max_tokens=3000, top_k_initial=20, top_k_final=5)
    return normal_rag_answer(question, retrieved, max_tokens=max_tokens)

def extract_pdf_text(pdf_name: str, pdf_text_json_path: str) -> str:
    """Extract text for a given PDF from JSON."""
    try:
        with open(pdf_text_json_path, 'r') as f:
            pdf_data = json.load(f)
    except FileNotFoundError:
        logging.error(f"PDF text JSON file not found: {pdf_text_json_path}")
        return ""
    except json.JSONDecodeError:
        logging.error(f"Invalid JSON in {pdf_text_json_path}")
        return ""

    if pdf_name not in pdf_data:
        logging.error(f"PDF {pdf_name} not found in {pdf_text_json_path}")
        return ""

    text_data = pdf_data.get(pdf_name, {})
    text = text_data.get("text", "")
    if not text:
        logging.warning(f"No text found for PDF {pdf_name}")
    return text

def generate_rag_answers(reference_json_path: str, pdf_text_json_path: str, output_json_path: str) -> None:
    """Generate RAG answers for all questions and save to JSON."""
    try:
        with open(reference_json_path, 'r') as f:
            reference_data = json.load(f)
    except FileNotFoundError:
        logging.error(f"Reference JSON file not found: {reference_json_path}")
        return
    except json.JSONDecodeError:
        logging.error(f"Invalid JSON in {reference_json_path}")
        return

    rag_answers = {}

    for pdf_name, entries in reference_data.items():
        logging.info(f"Processing PDF: {pdf_name}")
        pdf_text = extract_pdf_text(pdf_name, pdf_text_json_path)
        if not pdf_text:
            logging.warning(f"Skipping PDF {pdf_name} due to missing text")
            continue

        chunks = build_corpus_and_chunks(pdf_text)
        if not chunks:
            logging.warning(f"No chunks created for PDF {pdf_name}")
            continue

        rag_entries = []
        for entry in entries:
            question = entry.get("question", "")
            if not question:
                logging.warning(f"Skipping empty question in PDF {pdf_name}")
                continue

            answer = get_rag_answer_from_chunks(question, chunks)
            if len(answer) < 50:  # Arbitrary threshold for incomplete answers
                logging.warning(f"Short answer generated for question: {question[:50]}...")
            rag_entries.append({"question": question, "answer": answer})

        rag_answers[pdf_name] = rag_entries

    try:
        with open(output_json_path, 'w') as f:
            json.dump(rag_answers, f, indent=2)
        logging.info(f"RAG answers saved to {output_json_path}")
    except Exception as e:
        logging.error(f"Error saving RAG answers: {e}")

if __name__ == "__main__":
    # Define file paths
    reference_json_path = "/content/question_answers.json"  # e.g., {"vision_pro": [{"question": "..."}]}
    pdf_text_json_path = "/content/pdf_text_data.json"     # e.g., {"vision_pro": {"text": "..."}}
    output_json_path = "/content/generated_rag_answers.json"  # Output path

    # Run the pipeline
    generate_rag_answers(reference_json_path, pdf_text_json_path, output_json_path)